<a href="https://colab.research.google.com/github/jai2shan/TSAI-EVA40-Assignments/blob/master/Session%202/EVA4_Session_2_My_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)     #input Image Size - 28; Output size- 28, Receptive field (for Output) - 3, Kernels - 32
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)    #input Image Size - 28(Hypothetical - 26); Output size- 28, Receptive field (for Output) - 5 , Kernels - 64
        self.pool1 = nn.MaxPool2d(2, 2)                 #input Image Size - 28; Output size- 14, Receptive field (for Output) - 5, Kernels 
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)   #input image size - 14; Output size- 14, Receptive field (for Output) - 7, Kernels - 128
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)  #input image size - 14; Output size- 14, Receptive field (for Output) - 9, Kernels - 256
        self.pool2 = nn.MaxPool2d(2, 2)                 #input image size - 14; Output size- 7, Receptive field (for Output) - 11, 
        self.conv5 = nn.Conv2d(256, 512, 3)             #input image size - 7; Output size- 7, Receptive field (for Output) - 13, Kernels - 512
        self.conv6 = nn.Conv2d(512, 1024, 3)            #input image size - 7; Output size- 7, Receptive field (for Output) - 15, Kernels - 1024
        self.conv7 = nn.Conv2d(1024, 10, 3)             #input image size - 7; Output size- 7, Receptive field (for Output) - 17, Kernels - 10

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # Block 1 : conv1 -> Relu -> conv2 -> Relu -> Max Pooling ->  : Block 2
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) # Block 2 : conv3 -> Relu -> conv4 -> Relu -> Max Pooling ->  : Block 3
        x = F.relu(self.conv6(F.relu(self.conv5(x))))             # Block 3 : conv5 -> Relu -> conv6 -> Relu -> : Block 4 
        x = F.relu(self.conv7(x))                                 # Block 4 : conv7
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [4]:
#!pip install torchsummary
# Loading the Torch summary to display the model summary
from torchsummary import summary
# Check if cuda is available for processing
use_cuda = torch.cuda.is_available()
# if available use the GPU(cuda) else CPU
device = torch.device("cuda" if use_cuda else "cpu")
# Initiate the model the available device
model = Net().to(device)
# Print the summary of the model
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [5]:

# Setting seed so that the results are replicable
torch.manual_seed(1)
#
batch_size = 128

# Performing transformations on the data 

transform__=transforms.Compose([
                        transforms.ToTensor(), ## Converting the images into tensors
                        transforms.Normalize((0.1307,), (0.3081,)) ## Normalizing the values in the images
                    ])

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
                                            datasets.MNIST('../data', train=True, download=True,transform = transform__), # Chosing MNIST Dataset and performing transformations
                                            batch_size=batch_size, 
                                            shuffle=True, 
                                            **kwargs)


test_loader = torch.utils.data.DataLoader(
                                            datasets.MNIST('../data', train=False, transform = transform__),
                                            batch_size=batch_size, 
                                            shuffle=True, 
                                            **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()  # Training the model
    pbar = tqdm(train_loader) # Progress bar
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device) # Move the data to device
        optimizer.zero_grad() # Calling Optimizer 
        output = model(data) # Passing data to model
        loss = F.nll_loss(output, target) # Calculating Loss
        loss.backward() # Backward propagation
        optimizer.step() # Updating Learning Rate
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}') # print 


def test(model, device, test_loader):
    model.eval() # Evaluate model on test data
    test_loss = 0 # Initiate Loss as 0
    correct = 0 # Initiate Correct predictions
    with torch.no_grad(): 
        for data, target in test_loader:
            data, target = data.to(device), target.to(device) # Send the data to device
            output = model(data) # Predict with built model
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item() # if prediction is correct add the loss

    test_loss /= len(test_loader.dataset) # test loss = testloss/length of test dataset i.e, Average Loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), # print the loss value
        100. * correct / len(test_loader.dataset))) # Print the Accuracy

In [9]:

model = Net().to(device) # Run the model on device
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch) 
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.2941, Accuracy: 8832/10000 (88%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0473, Accuracy: 9846/10000 (98%)



loss=0.008363448083400726 batch_id=352:  74%|███████▍  | 349/469 [00:08<00:02, 41.98it/s]

KeyboardInterrupt: ignored